# Polish NLProc #6 - spaCy mówi po polsku, 6 listopada 2020


Ryszard Tuora


2 modele do języka polskiego w spaCy:

IPI PAN - bardziej rozbudowany, wolniejszy, bardziej złożona instalacja - http://zil.ipipan.waw.pl/SpacyPL
Więcej informacji: https://github.com/ipipan/spacy-pl

model oficjalny - prostszy, znacznie szybszy, prosta instalacja - https://spacy.io/models/pl

model IPI PAN dla języka polskiego

składa się z:
- taggera morfosyntaktycznego
- lematyzatora
- parsera zależnościowego
- komponentu NER
- flexera (komponentu do fleksji)

In [ ]:
# Przygotowanie środowiska, komendy linux
# instalacja Morfeusza 2
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2


# instalacja spaCy

!python3 -m pip install spacy

# 1. instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz' pl_spacy_model_morfeusz-0.1.3.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.3.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

# 2. instalacja oficjalnego modelu spaCy
!python3 -m spacy download pl_core_news_lg


In [1]:
### PYTHON 3
# ładowanie modelu
import spacy
import requests

nlp = spacy.load("pl_spacy_model_morfeusz") # IPI PAN
#nlp = spacy.load("pl_core_news_lg") # OFICJALNY

# Część zero - Tokenizacja i reprezentacja tekstów

Wejściem do modelu są łańcuchy tekstowe (stringi), na wyjściu dostajemy obiekt Doc reprezentujący struktury wykryte w tekście przez pełen potok (pipeline). Pierwszym krokiem który musi być wykonany aby przetworztć tekst, jest tokenizacja, czyli podział tekstu na tokeny/segmenty. Tokeny w większości przypadków odpowiadają słowom "od spacji do spacji". Ale warto zwrócić uwagę na kilka przypadków odstających od takiej prostej reguły.

Wynikiem potoku spaCy jest obiekt Doc, który składa się z obiektów Token.

In [ ]:
txt = "Chciałby, żebym pojechał do miasta z zielono-żółto-białą flagą (np. Zielonej Góry)."
split = txt.split()
doc = nlp(txt)
print("    spaCy     vs.   .split()\n")
for i in range(max([len(split), len(doc)])):
    try:
        tok1 = doc[i]
    except IndexError:
        tok1 = ""
    try:
        tok2 = split[i]
    except IndexError:
        tok2 = ""
    print("{0:2}. {1:15} {2:15}".format(i, tok1.orth_, tok2))

spaCy pozwala na zapisywanie przetworzonych dokumentów w formacie JSON

In [ ]:
import json

doc_json = doc.to_json()
json_string = json.dumps(doc_json, indent=2, ensure_ascii=False)
print(json_string)

# Część pierwsza - reprezentacje wektorowe

Sercem modelu jest reprezentacja wektorowa słów, 500 tys. wektorów o długości 100 wyekstrahowanych z embeddingów word2vec KGR10.

### Bardzo popularny przykład "arytmetyki słów":
znaczenie słów jest reprezentowane przez wektory, dla których mamy zdefiniowane operacje matematyczne. Możemy więc odjąć od znaczenia słowa "królowa", znaczenie słowa "kobieta", i dodać doń znaczenie słowa "mężczyzna", licząc iż wektor będący wynikiem takiego działania odpowiada słowu "król". W praktyce wektor taki najprawdopodobniej nie ma interpretacji, ale możemy znaleźć najbliższy wektor który ma jakąkolwiek interpretację przeszukując słownik.

In [ ]:
# od wersji 2.3 wektory są ładowane "leniwie"
# musimy więc najpierw wymusić załadowanie wszystkich wektorów
print(len(nlp.vocab))
for s in nlp.vocab.vectors:
    _ = nlp.vocab[s]
print(len(nlp.vocab))

In [ ]:
from scipy import spatial
 
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)
 
man = nlp.vocab['mężczyzna'].vector
woman = nlp.vocab['kobieta'].vector
queen = nlp.vocab['królowa'].vector
king = nlp.vocab['król'].vector
 

maybe_king = man - woman + queen
computed_similarities = []
 
for word in nlp.vocab:
    # Ignore words without vectors
    if not word.has_vector:
        continue 
    similarity = cosine_similarity(maybe_king, word.vector)
    computed_similarities.append((word, similarity))
 
sorted_similarities = sorted(computed_similarities, key=lambda item: -item[1])
print([w[0].text for w in sorted_similarities[:10]])

spaCy konstruuje także wektory dla zdań, i dokumentów. Wektor dla dokumentu jest dostępny w atrybucie .vector obiektu Doc, i jest równy uśrednionemu wektorowi tokenów z tego dokumentu.

In [ ]:
document_vector = doc.vector
mean_document_vector = sum([tok.vector for tok in doc])/len(doc)
print(document_vector == mean_document_vector)

# Część druga - Tagowanie morfosyntaktyczne
korzystamy z tagsetu NKJP
Nasz tagger to słownikowy tagger Morfeusz2 + dezambiguacja za pomocą neuronowego Toyggera (biLSTM)

Każdy token t ma trzy interesujące nas atrybuty: 
- t.tag_ : klasa gramatyczna według polskiego tagsetu NKJP (http://nkjp.pl/poliqarp/help/ense2.html)
- t.pos_ : klasa gramatyczna według międzynarodowego tagsetu UD (mapowana z NKJP)
- t._.feats : customowy atrybut odpowiadający cechom morfosyntaktycznym (np. rodzajowi gramatycznemu, lub liczbie), poszczególne wartości cech są oddzielone dwukropkiem

In [ ]:
txt = "Nornica prowadzi zmierzchowo-nocny tryb życia, ale wychodzi również za dnia w poszukiwaniu pokarmu."
doc = nlp(txt) # przetworzenie textu przez pipeline, na wyjściu dostajemy iterowalny obiekty klasy Doc, przechowujący tokeny

print("{0:15} {1:8} {2:6} {3:15}\n".format(".orth_", "NKJP", "UD POS", "._.feats"))
for t in doc:
    print("{0:15} {1:8} {2:6} {3:15}".format(t.orth_, t.tag_, t.pos_, t._.feats)) # wypisujemy interpretację morfosyntaktyczną każdego tokenu

### Fleksja

Flexer pozwala na odmianę pojedynczych tokenów, do pożądanej charakterystyki morfologicznej. Argumentami do flexera jest słowo do odmiany (string, lub lepiej otagowany token), i przedzielony dwukropkami string znaczników morfosyntaktycznych.

Flexer umożliwia np. wypełnianie szablonów tekstów.

In [ ]:
filizanka = "filiżanka"
flexer = nlp.get_pipe("flexer")

tmpl1 = "Szukam szarej, ceramicznej {}.".format(flexer.flex(filizanka, "gen"))
tmpl2 = "Marzę o szarej, ceramicznej {}.".format(flexer.flex(filizanka, "loc"))
tmpl3 = "Szukam kompletu ceramicznych {}.".format(flexer.flex(filizanka, "gen:pl"))
print(tmpl1)
print(tmpl2)
print(tmpl3)

#### Flexer pozwala także na odmianę fraz wielowyrazowych (multi-word expressions - MWE).

W tym celu możemy do niego podać string, lub token odpowiadający głowie frazy.
Korzystamy z metody .flex_mwe()

In [ ]:
filizanka2 = "biała, porcelanowa filiżanka z Chin"

tmpl1 = "Szukam {}.".format(flexer.flex_mwe(filizanka2, "gen"))
tmpl2 = "Marzę o {}.".format(flexer.flex_mwe(filizanka2, "loc"))
tmpl3 = "Szukam kompletu {}.".format(flexer.flex_mwe(filizanka2, "gen:pl"))

print(tmpl1)
print(tmpl2)
print(tmpl3)

# Część trzecia - Lematyzacja i własności leksykalne
Nasz model umożliwia słownikową lematyzację przy pomocy Morfeusza, do dezambiguacji (tutaj np. rozróżnienia między "Głos zabrały mamy dzieci."-> "mama" i "My mamy samochód." -> "mieć" służy output taggera).

Lematyzacja pozwala redukować redundancję informacyjną, i ułatwiać zadania takie jak streszczanie, i przeszukiwanie.

Każdy z tokenów dodatkowo jest oznaczony ze względu na pewne własności leksykalne, np. :
- t.lemma_ - lemat - forma reprezentatywna danego leksemu
- t.is_stop - słowo należy do stoplisty (listy słów występujących najczęściej, a więc najmniej istotnych semantycznie)
- t.is_oov - słowo znajduje się poza słownikiem, i.e. embeddingami wykorzystanymi w modelu
- t.like_url - token ma strukturę url-a
- t.like_num - token jest liczbą
- t.is_alpha - token składa się tylko ze znaków alfabetycznych
- t.rank - miejse w rankingu częstości słów

In [ ]:
txt = "Zdenerwowany gen. Leese mówił przez telefon swym podwładnym walczącym pod Monte Cassino, że rozmawia z nimi ze schronu."
doc = nlp(txt)
print("{0:16} {1:16} {2:5} {3:5} {4:20}\n".format("forma", "lemat", "OOV", "STOP", "Częstość"))
for t in doc:
    print("{0:16} {1:16} {2:5} {3:5} {4:20}".format(t.orth_, t.lemma_, t.is_oov, t.is_stop, t.rank)) # orth_ to atrybut odpowiadający formie słowa występującej w tekście

# Część czwarta - parsowanie zależnościowe

spaCy zawiera parser zależnościowy oparty o metodologię transition-based dependency parsing. 
Interesują nas tu cztery atrybuty:
 - t.head - link do tokenu będącego nadrzędnikiem tokenu t
 - t.dep_ - etykieta opisująca rodzaj zależności
 - t.subtree - generator opisujący poddrzewo rozpięte przez token t
 - t.children - generator opisujący wszystkich bezpośrednich potomków tokenu t
 - t.ancestors - generator opisujący wszystkie przechodnie nadrzędniki tokenu t


Opis systemu etykiet: https://universaldependencies.org/u/dep/all.html

In [ ]:
txt = "Pierwsza wzmianka o Gdańsku pochodzi ze spisanego po łacinie w 999 Żywotu świętego Wojciecha."
doc = nlp(txt)

import pandas as pd

table = []
for tok in doc:
    tok_dic = {"form": tok.orth_, "label": tok.dep_, "head": tok.head.orth_, "subtree": list(tok.subtree), "ancestors": list(tok.ancestors)}
    table.append(tok_dic)
df = pd.DataFrame(table)
print(df.to_string())

### spaCy posiada wbudowaną wizualizację drzew zależnościowych

In [ ]:
from spacy import displacy

displacy.render(doc, jupyter = True)

### Poniższa funkcja służy łatwej wizualizacji podstawowych własności tokenów z danego tekstu

In [ ]:
import pandas as pd

def table(doc):
    table = []
    for tok in doc:
        tok_dic = {"form": tok.orth_, "lemma": tok.lemma_, "tag": ":".join([tok.tag_, tok._.feats]), "dep_label": tok.dep_, "dep_head": tok.head.orth_}
        table.append(tok_dic)
    return pd.DataFrame(table)

txt = "Wiadomość jest symboliczna, ale oznacza też początek długotrwałego trendu.\
 Dochód na mieszkańca z uwzględnieniem realnej mocy nabywczej walut narodowych \
 wyniósł w 2019 r. w Rzeczpospolitej Polskiej 33 891 dolarów, nieco więcej, niż w Portugalii \
 (33 665 dolarów). Jednak Fundusz przewiduje, że w tym roku portugalska gospodarka\
  będzie się rozwijać w tempie 1,6 proc. wobec 3,1 proc. w przypadku gospodarki \
  polskiej. Nożyce między oboma krajami będą się więc rozwierać."

doc = nlp(txt)

tab = table(doc)

print(tab.to_string()) # prosty hack na wypisanie całej tabeli

### Parser zależnościowy pozwala także na dzielenie dokumentów na zdania w sposób bardziej inteligentny, niż posługując się regułami interpunkcji.
Za zdanie uważamy nieprzerwaną sekwencję tokenów które są powiązane relacjami zależnościowymi.
Zdania są zapisane w atrybucie doc.sents dokumentu.

In [ ]:
for s in doc.sents:
    print(s)

displacy.render(doc, jupyter = True)

# Część czwarta - Rozpoznawanie jednostek nazewniczych (NER)
#### Nasz model do spaCy wykorzystuje 6 rodzajów etykiet:
- placeName - miejsca antropogeniczne, np. Dania, Londyn
- geogName - naturalne miejsca geograficzne, np. Tatry, Kreta
- persName - imiona i nazwiska osób, np. J. F. Kennedy, gen. Maczek
- orgName - nazwy organizacji, np. NATO, Unia Europejska
- date - daty, np. 22 marca 1999, druga połowa kwietnia
- time - czas, np. 18:55, pięć po dwunastej

#### Nie pozwala na wykrywanie zagnieżdżonych jednostek nazewniczych, np. [placeName: **aleja** [persName: **Piłsudskiego**]]

Wykryte wzmianki są przechowywane w atrybucie doc.ents dokumentu, każda z tych wzmianek ma atrybut ent.label_, w którym przechowywana jest jej etykieta.

In [ ]:
print(doc, "\n\n")
for e in doc.ents:
    print(e, e.label_)


### displaCy pozwala także na wizualizację jednostek nazewniczych

In [ ]:
displacy.render(doc, style="ent", jupyter = True)

W obecnym modelu NER, uwzględnione są "uniwersalne" kategorie jednostek nazewniczych, jednak w zależności od zastosowania będziemy najprawdopodobniej potrzebowali innych kategorii - np. osobnej kategorii dla nazw aktów prawodawczych, lub kwot i walut.

Bez problemu można zastąpić domyślny model własnym, wytrenowanym przez CLI spaCy na własnych oznakowanych danych. Więcej informacji tu: https://spacy.io/api/cli#train

Jednostki które da się wykryć na podstawie reguł, można wykrywać poprzez EntityRuler: https://spacy.io/api/entityruler